In [1]:
!pip install diffusers #use diffusers auth token in your colab environment for quick and better performance of code. 
# !pip install transformers scipy ftfy accelerate #to speed up the computations
# !pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html # optional to speed up ur GPU 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.5 MB/s eta 0:00:00


In [2]:
import torch
import cv2
import numpy as np
from google.colab.patches import cv2_imshow #since We used google Colab for this project we are using this extension
from diffusers import DiffusionPipeline

# pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16) #some of the experimented diffuser models
# pipe = pipe.to("cuda")

# pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",torch_dtype=torch.float16)
# pipe = pipe.to("cuda")


pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16") # you can use float 32 if you prefer
pipe.to("cuda:0")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

text_encoder_2/config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.29.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [4]:
!pip install pyngrok #including NGROK to our python 
!pip install flask-ngrok #to give web access to our flask from Colab 
!pip install flask

In [5]:

from google.colab import files #use our background image attached 

uploaded = files.upload()


Saving loegbmrn.png to loegbmrn.png


In [6]:
import os
import shutil

# Create a static directory if it doesn't exist
os.makedirs('static', exist_ok=True) 

# Move the uploaded image to the static directory
for filename in uploaded.keys():
    shutil.move(filename, 'static/background.jpg')  # You can change the destination filename if needed


In [ ]:
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import base64

# Define the generate function using OpenCV 
def generate(prompt):
    image = pipe(prompt).images[0]
    image_array = np.array(image)
    image_bgr = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR) #since our organization instructed us to compulsorily use OpenCV we are using this method

    #Encode image to base64
    #Converts the generated image to an OpenCV format.
    #Uses OpenCV to convert the RGB image to BGR format (which OpenCV uses).
    #Encodes the image to PNG format and then to a base64 string.

    _, buffer = cv2.imencode('.png', image_bgr)
    img_str = base64.b64encode(buffer).decode('utf-8')
    return img_str

# Create the Flask app
app = Flask(__name__)

@app.route('/')
def home(): # since it is not possible to upload a file in colab each and every time we run code to make it easily accicible we are using an integrated code here:
    return render_template_string('''
  <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text to Image Generator</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-image: url('/static/background.jpg');
            background-size: cover;
            background-position: center;
            display: flex;
            flex-direction: column;
            height: 100vh;
            margin: 0;
            color: #fff;
             /* Disable scrolling */
        }
            .header-container {
            border-radius: 10px;
            padding: 10px; /* Reduce padding */
            width: 50%;
            max-width: 800px;
            background: linear-gradient(145deg, #070649, #0d0d0d);
            box-shadow: 0 0 20px rgba(5, 14, 148, 0.5);
           margin: 60px auto 20px;
            text-align: center;
            margin-left: 300px;
        }
     h1 {
            font-size: 3rem;
            background: linear-gradient(#fff, #fff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            text-shadow: 0 0 10px rgba(44, 12, 224, 0.5);
            margin: 0; /* Remove margin to reduce height */
        }
        .form-container {
          display: flex;
            justify-content: center; /* Center horizontally */
            align-items: center; /* Center vertically */
            margin-top: 10px;


        }
        .textForm {
            margin: 50px auto 10px;
            text-align: center;
            margin-right: 60%;
            margin-top: 15px;
            margin-left: 400px;
        }
        .abc{
           margin: 40px auto 20px;
            text-align: center;
            margin-right: 50%;
            margin-top: 15px;
            margin-left: 400px;

        }
        .gen {
            display: flex;
            flex-direction: column;
            align-items: center;
            padding: 20px;
            margin-left: 80%;
            margin-left: 500px;
            margin-top: 230px;
        }
        label {
            font-weight: bold;
            margin-bottom: 10px;
            font-size: 1.2rem;
            color: #fff;
            text-shadow: 0 0 10px rgb(29, 16, 211);
            border: 2px solid rgb(20, 120, 235);
            padding: 5px 10px;
            border-radius: 5px;
            background: black;
            background-image: radial-gradient(circle, rgb(25, 157, 218) 1px, transparent 1px), radial-gradient(circle, rgb(87, 174, 231) 1px, transparent 1px);
            background-position: 0 0, 5px 5px;
            background-size: 10px 10px;
            margin-right: 10px; /* Add space between label and input */
        }
        input[type="text"] {
            padding: 12px;
            margin-bottom: 20px;
            width: 5cm; /* Width set to 5cm */
            border: 2px solid rgb(72, 182, 233);
            border-radius: 5px;
            font-size: 1rem;
            transition: border-color 0.3s ease;
            background: black;
            color: rgb(25, 0, 255);
            box-shadow: inset 0 0 10px rgb(16, 31, 236);
            text-align: center; /* Center the text inside the input */
            /* Remove centering the input box */
        }
        input[type="text"]:focus {
            border-color: #4CAF50;
            box-shadow: 0 0 10px #4CAF50;
        }
        button[type="submit"] {
            padding: 15px 30px;
            background-color: #333;
            color: white;
            border: 3px solid #555;
            border-radius: 50px;
            cursor: pointer;
            font-size: 1rem;
            transition: background-color 0.3s ease, transform 0.3s ease;
            margin-top: 20px;
            box-shadow: 0 4px #555, 0 4px 10px rgba(0, 0, 0, 0.2);
            width: 200px;
            text-transform: uppercase;
            font-weight: bold;
            position: relative;
            overflow: hidden;
            outline: none;
            z-index: 1;
            background-image: linear-gradient(45deg, #4390cf, #1408c7);
        }
        button[type="submit"]:before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            width: 0%;
            height: 100%;
            background-color: rgba(255, 255, 255, 0.1);
            transition: width 0.3s ease;
            z-index: -1;
        }
        button[type="submit"]:hover:before {
            width: 100%;
        }
        button[type="submit"]:hover {
            background-color: #444;
            color: #fff;
            border: 3px solid #fff;
            box-shadow: 0 4px #fff, 0 4px 10px rgba(0, 0, 0, 0.2);
        }
        button[type="submit"]:active {
            background-color: #555;
            box-shadow: 0 2px #555, 0 2px 5px rgba(0, 0, 0, 0.2);
            transform: translateY(2px);
        }
        button[type="submit"]::after {
            content: '';
            position: absolute;
            width: 100%;
            height: 100%;
            background: radial-gradient(circle, #1747e4 10%, transparent 10.01%);
            background-size: 1000% 1000%;
            top: 0;
            left: 0;
            animation: anim 1s infinite linear;
        }

        @keyframes anim {
            to {
                background-position: 100% 100%;
            }
        }

        #imageContainer {
            opacity: 0;
            transition: opacity 0.5s ease-in-out;
            position: fixed;
        }
        #imageContainer.show {
            opacity: 1;
            position: fixed;
        }
        img {
            width: 300px;
            height: 300px;
            margin-right: 20%;
            margin-top: 150px;
            margin-left: 600px;
            transform: translate(-50%, -50%);
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease;
        }

        #spinner {
            display: none;
            position: fixed;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            width: 50px;
            height: 50px;
            border: 3px solid transparent;
            border-top: 3px solid #3498db;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }


    </style>
</head>
<body>

    <div class="header-container">
        <h1>Text to Image Generator</h1>
    </div>

    <form id="textForm">
        <div class="form-container">
            <label for="textInput">Enter Text:</label>
            <input type="text" id="textInput" name="text">
        </div>
        <div id="spinner"></div>

        <div class="image-container" id="imageContainer"></div>

        <div class="gen">
            <button type="submit">Generate Image</button>
        </div>
    </form>

    <script>
        document.getElementById('textForm').addEventListener('submit', function(event) {
            event.preventDefault();
            var textInput = document.getElementById('textInput').value;

            var imageContainer = document.getElementById('imageContainer');
            imageContainer.innerHTML = '';
            imageContainer.classList.remove('show');

            var spinner = document.getElementById('spinner');
            spinner.style.display = 'block';
            document.body.classList.add('no-scroll');

            fetch('/generate', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ text: textInput })
            })
            .then(response => response.json())
            .then(data => {
                var img = document.createElement('img');
                img.src = 'data:image/png;base64,' + data.image;
                imageContainer.appendChild(img);
                imageContainer.classList.add('show');

                spinner.style.display = 'none';
                document.body.classList.remove('no-scroll');
            })
            .catch(error => {
                console.error('Error:', error);

                spinner.style.display = 'none';
                document.body.classList.remove('no-scroll');
            });
        });
    </script>
</body>
</html>


    ''')

@app.route('/generate', methods=['POST'])
def generate_image():
    data = request.json
    text = data['text']
    # Call the generate function to create an image from the text
    image = generate(text)
    if image is None:
        return jsonify({'error': 'Failed to generate image'}), 500
    return jsonify({'image': image})

# Start ngrok when the app runs
ngrok.set_auth_token("2c5uitQ0zI4opvjlnWdHU4NxQVi_2raMEo2tg4pC2PHTU9Z6q")  # Replace with your ngrok auth token this tocken is unavailable now... 
public_url = ngrok.connect(5000)
print('Public URL:', public_url)

if __name__ == '__main__':
    app.run(port=5000)

Public URL: NgrokTunnel: "https://592d-34-125-42-158.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Aug/2024 11:43:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Aug/2024 11:43:23] "GET /static/background.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Aug/2024 11:43:24] "GET /favicon.ico HTTP/1.1" 404 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [04/Aug/2024 11:44:22] "POST /generate HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

# Thank You 